In [ ]:
#@title 🚀 SHA-256 BACKDOOR DISCOVERY - GPU ACCELERATED

"""
SHA-256 BACKDOOR DISCOVERY ENGINE - GPU ACCELERATED
=====================================================

Uses PyTorch/CUDA for massive parallelization:
- Batch SHA-256 computation on GPU
- Parallel population evaluation
- Vectorized fitness calculation
- 100-1000x speedup over CPU

Expected: Evaluate thousands of conditions per second
"""

!pip install python-sat torch -q

import numpy as np
import torch
import torch.nn.functional as F
import hashlib
import time
import random
import json
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
import os

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("="*70)
print("  🚀 SHA-256 BACKDOOR DISCOVERY - GPU ACCELERATED")
print("="*70)
print(f"  Device: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("  ⚠️  No GPU detected - will use CPU (slower)")
print("="*70)

# ============================================================
# SHA-256 CONSTANTS (as torch tensors on GPU)
# ============================================================

K_CONSTANTS = torch.tensor([
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
], dtype=torch.int64, device=device)

H_INIT = torch.tensor([
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
], dtype=torch.int64, device=device)

MASK32 = 0xFFFFFFFF


# ============================================================
# GPU-ACCELERATED SHA-256
# ============================================================

class SHA256_GPU:
    """
    Vectorized SHA-256 implementation on GPU.
    Processes multiple messages in parallel.
    """

    @staticmethod
    def rotr(x, n):
        """Rotate right (circular shift)"""
        return ((x >> n) | (x << (32 - n))) & MASK32

    @staticmethod
    def shr(x, n):
        """Shift right"""
        return x >> n

    @staticmethod
    def ch(e, f, g):
        """Choice function"""
        return (e & f) ^ ((~e) & g) & MASK32

    @staticmethod
    def maj(a, b, c):
        """Majority function"""
        return (a & b) ^ (a & c) ^ (b & c)

    @staticmethod
    def sigma0(x):
        """σ0 function"""
        return SHA256_GPU.rotr(x, 7) ^ SHA256_GPU.rotr(x, 18) ^ SHA256_GPU.shr(x, 3)

    @staticmethod
    def sigma1(x):
        """σ1 function"""
        return SHA256_GPU.rotr(x, 17) ^ SHA256_GPU.rotr(x, 19) ^ SHA256_GPU.shr(x, 10)

    @staticmethod
    def Sigma0(x):
        """Σ0 function"""
        return SHA256_GPU.rotr(x, 2) ^ SHA256_GPU.rotr(x, 13) ^ SHA256_GPU.rotr(x, 22)

    @staticmethod
    def Sigma1(x):
        """Σ1 function"""
        return SHA256_GPU.rotr(x, 6) ^ SHA256_GPU.rotr(x, 11) ^ SHA256_GPU.rotr(x, 25)

    @staticmethod
    def hash_batch(messages):
        """
        Hash a batch of messages on GPU.

        Args:
            messages: Tensor of shape (batch_size, 16) containing 32-bit words

        Returns:
            Tensor of shape (batch_size, 8) containing hash words
        """
        batch_size = messages.shape[0]

        # Ensure on correct device
        messages = messages.to(device).long()

        # Message schedule
        W = torch.zeros((batch_size, 64), dtype=torch.int64, device=device)
        W[:, :16] = messages & MASK32

        for t in range(16, 64):
            s0 = SHA256_GPU.sigma0(W[:, t-15])
            s1 = SHA256_GPU.sigma1(W[:, t-2])
            W[:, t] = (W[:, t-16] + s0 + W[:, t-7] + s1) & MASK32

        # Initialize working variables
        a = H_INIT[0].expand(batch_size).clone()
        b = H_INIT[1].expand(batch_size).clone()
        c = H_INIT[2].expand(batch_size).clone()
        d = H_INIT[3].expand(batch_size).clone()
        e = H_INIT[4].expand(batch_size).clone()
        f = H_INIT[5].expand(batch_size).clone()
        g = H_INIT[6].expand(batch_size).clone()
        h = H_INIT[7].expand(batch_size).clone()

        # Compression
        for t in range(64):
            S1 = SHA256_GPU.Sigma1(e)
            ch = SHA256_GPU.ch(e, f, g)
            temp1 = (h + S1 + ch + K_CONSTANTS[t] + W[:, t]) & MASK32
            S0 = SHA256_GPU.Sigma0(a)
            maj = SHA256_GPU.maj(a, b, c)
            temp2 = (S0 + maj) & MASK32

            h = g
            g = f
            f = e
            e = (d + temp1) & MASK32
            d = c
            c = b
            b = a
            a = (temp1 + temp2) & MASK32

        # Final hash
        result = torch.stack([
            (a + H_INIT[0]) & MASK32,
            (b + H_INIT[1]) & MASK32,
            (c + H_INIT[2]) & MASK32,
            (d + H_INIT[3]) & MASK32,
            (e + H_INIT[4]) & MASK32,
            (f + H_INIT[5]) & MASK32,
            (g + H_INIT[6]) & MASK32,
            (h + H_INIT[7]) & MASK32,
        ], dim=1)

        return result

    @staticmethod
    def count_matching_bits_batch(hash_a, hash_b):
        """
        Count matching bits between two batches of hashes.

        Returns: Tensor of shape (batch_size,) with matching bit counts
        """
        # XOR to find differences
        xor = hash_a ^ hash_b

        # Count zeros (matching bits) using popcount
        # PyTorch doesn't have popcount, so we do it manually
        matching = torch.zeros(xor.shape[0], dtype=torch.int64, device=device)

        for word in range(8):
            word_xor = xor[:, word]
            # Count set bits in XOR (these are different bits)
            diff_bits = torch.zeros_like(word_xor)
            temp = word_xor
            for _ in range(32):
                diff_bits += temp & 1
                temp = temp >> 1
            matching += 32 - diff_bits

        return matching


# ============================================================
# FAST BATCH DIFFERENTIAL TESTER
# ============================================================

class BatchDifferentialTester:
    """
    GPU-accelerated batch testing of backdoor conditions.
    Tests hundreds/thousands of message pairs simultaneously.
    """

    def __init__(self, batch_size=1024):
        self.batch_size = batch_size

    def bits_to_words(self, bits_batch):
        """
        Convert batch of 512 bits to 16 words.
        bits_batch: (batch_size, 512) tensor
        Returns: (batch_size, 16) tensor of 32-bit words
        """
        batch_size = bits_batch.shape[0]
        words = torch.zeros((batch_size, 16), dtype=torch.int64, device=device)

        for w in range(16):
            word_bits = bits_batch[:, w*32:(w+1)*32]
            # Convert bits to word
            for b in range(32):
                words[:, w] += word_bits[:, b].long() << b

        return words

    def generate_messages_batch(self, msg_constraints, diff_constraints, batch_size):
        """
        Generate batch of message pairs satisfying constraints.

        Returns: (messages_A, messages_B) as word tensors
        """
        # Generate random bits
        A_bits = torch.randint(0, 2, (batch_size, 512), dtype=torch.int64, device=device)

        # Apply message constraints
        constraint_mask = torch.tensor(msg_constraints, device=device)

        # Where constraint == 1, force to 0
        zero_mask = (constraint_mask == 1)
        A_bits[:, zero_mask] = 0

        # Where constraint == 2, force to 1
        one_mask = (constraint_mask == 2)
        A_bits[:, one_mask] = 1

        # Generate differential
        diff_bits = torch.zeros((batch_size, 512), dtype=torch.int64, device=device)
        diff_mask = torch.tensor(diff_constraints, device=device)

        # Where diff_constraint == 2, set to 1
        diff_active = (diff_mask == 2)
        diff_bits[:, diff_active] = 1

        # Ensure at least one diff bit
        if diff_bits.sum() == 0:
            diff_bits[:, 0] = 1

        # B = A XOR diff
        B_bits = A_bits ^ diff_bits

        # Convert to words
        A_words = self.bits_to_words(A_bits)
        B_words = self.bits_to_words(B_bits)

        return A_words, B_words, A_bits, B_bits

    def test_condition(self, msg_constraints, diff_constraints, correlations=None, num_batches=10):
        """
        Test a backdoor condition with GPU acceleration.

        Returns: (best_matching, best_A, best_B, hash_A, hash_B)
        """
        best_matching = 0
        best_A = None
        best_B = None
        best_hash_A = None
        best_hash_B = None

        for _ in range(num_batches):
            # Generate batch
            A_words, B_words, A_bits, B_bits = self.generate_messages_batch(
                msg_constraints, diff_constraints, self.batch_size
            )

            # Hash both batches
            hash_A = SHA256_GPU.hash_batch(A_words)
            hash_B = SHA256_GPU.hash_batch(B_words)

            # Count matching bits
            matching = SHA256_GPU.count_matching_bits_batch(hash_A, hash_B)

            # Find best in batch
            batch_best_idx = matching.argmax().item()
            batch_best_matching = matching[batch_best_idx].item()

            if batch_best_matching > best_matching:
                best_matching = batch_best_matching

                # Extract best messages
                best_A_bits = A_bits[batch_best_idx].cpu().numpy()
                best_B_bits = B_bits[batch_best_idx].cpu().numpy()

                # Convert to bytes
                def bits_to_bytes(bits):
                    words = []
                    for w in range(16):
                        val = sum(int(bits[w*32 + b]) << b for b in range(32))
                        words.append(val)
                    return b''.join(w.to_bytes(4, 'big') for w in words)

                best_A = bits_to_bytes(best_A_bits)
                best_B = bits_to_bytes(best_B_bits)

                # Get hash hex
                best_hash_A = ''.join(f'{int(h):08x}' for h in hash_A[batch_best_idx].cpu().numpy())
                best_hash_B = ''.join(f'{int(h):08x}' for h in hash_B[batch_best_idx].cpu().numpy())

                if best_matching == 256:
                    break

            if best_matching == 256:
                break

        return best_matching, best_A, best_B, best_hash_A, best_hash_B


# ============================================================
# GPU-ACCELERATED BACKDOOR GENOME
# ============================================================

class BackdoorGenome:
    """Represents a potential backdoor condition"""

    def __init__(self, genome_size=512):
        self.genome_size = genome_size
        self.msg_constraints = np.zeros(genome_size, dtype=np.int8)
        self.diff_constraints = np.zeros(genome_size, dtype=np.int8)
        self.correlations = []

        self.fitness = 0
        self.matching_bits = 0
        self.is_collision = False
        self.msg_A = None
        self.msg_B = None
        self.hash_A = None
        self.hash_B = None

        self.generation = 0
        self.parent_fitness = 0

    def randomize(self, constraint_density=0.05, diff_density=0.01):
        """Create random genome"""
        for i in range(self.genome_size):
            if random.random() < constraint_density:
                self.msg_constraints[i] = random.choice([1, 2])

        num_diffs = max(1, int(self.genome_size * diff_density))
        diff_positions = random.sample(range(self.genome_size), min(num_diffs, 10))
        for pos in diff_positions:
            self.diff_constraints[pos] = 2

    def mutate(self, mutation_rate=0.1, generation=0):
        """Mutate genome"""
        child = BackdoorGenome(self.genome_size)
        child.msg_constraints = self.msg_constraints.copy()
        child.diff_constraints = self.diff_constraints.copy()
        child.correlations = self.correlations.copy()
        child.generation = generation
        child.parent_fitness = self.fitness

        adaptive_rate = mutation_rate * (1.0 + 0.5 * (256 - self.matching_bits) / 256)

        for i in range(self.genome_size):
            if random.random() < adaptive_rate:
                if self.msg_constraints[i] == 0:
                    child.msg_constraints[i] = random.choice([1, 2])
                elif random.random() < 0.3:
                    child.msg_constraints[i] = 0
                else:
                    child.msg_constraints[i] = 3 - self.msg_constraints[i]

        for i in range(self.genome_size):
            if random.random() < adaptive_rate * 0.5:
                if self.diff_constraints[i] == 0:
                    if random.random() < 0.1:
                        child.diff_constraints[i] = 2
                else:
                    if random.random() < 0.3:
                        child.diff_constraints[i] = 0

        return child

    def crossover(self, other, generation=0):
        """Crossover with another genome"""
        child = BackdoorGenome(self.genome_size)
        child.generation = generation
        child.parent_fitness = max(self.fitness, other.fitness)

        p1 = random.randint(0, self.genome_size - 1)
        p2 = random.randint(p1, self.genome_size)

        child.msg_constraints[:p1] = self.msg_constraints[:p1]
        child.msg_constraints[p1:p2] = other.msg_constraints[p1:p2]
        child.msg_constraints[p2:] = self.msg_constraints[p2:]

        if self.fitness >= other.fitness:
            child.diff_constraints = self.diff_constraints.copy()
        else:
            child.diff_constraints = other.diff_constraints.copy()

        return child

    def get_summary(self):
        """Human-readable summary"""
        return {
            'msg_zeros': int(np.sum(self.msg_constraints == 1)),
            'msg_ones': int(np.sum(self.msg_constraints == 2)),
            'diff_bits': int(np.sum(self.diff_constraints == 2)),
            'matching': self.matching_bits,
            'fitness': self.fitness
        }

    def to_dict(self):
        return {
            'msg_constraints': self.msg_constraints.tolist(),
            'diff_constraints': self.diff_constraints.tolist(),
            'matching_bits': self.matching_bits,
            'is_collision': self.is_collision,
            'msg_A': self.msg_A.hex() if self.msg_A else None,
            'msg_B': self.msg_B.hex() if self.msg_B else None,
            'hash_A': self.hash_A,
            'hash_B': self.hash_B
        }


# ============================================================
# GPU-ACCELERATED EVOLUTIONARY ENGINE
# ============================================================

class GPUBackdoorDiscovery:
    """
    GPU-accelerated evolutionary backdoor discovery.

    Key optimizations:
    1. Batch SHA-256 on GPU
    2. Parallel population evaluation
    3. Vectorized fitness calculation
    """

    def __init__(self, population_size=100, batch_size=2048):
        self.population_size = population_size
        self.batch_size = batch_size
        self.tester = BatchDifferentialTester(batch_size)

        self.population = []
        self.generation = 0
        self.best_ever = None
        self.history = []

        self.stats = {
            'generations': 0,
            'evaluations': 0,
            'hashes_computed': 0,
            'best_matching': 0,
            'collisions_found': 0
        }

    def initialize_population(self):
        """Create diverse initial population"""
        print("\n  Initializing population...")
        self.population = []

        # Strategy mix for diversity
        strategies = [
            (0.02, 0.002),  # Very sparse
            (0.05, 0.002),  # Sparse
            (0.10, 0.003),  # Medium
            (0.15, 0.004),  # Dense
            (0.08, 0.005),  # More diffs
            (0.12, 0.002),  # Dense msg, sparse diff
        ]

        per_strategy = self.population_size // len(strategies)

        for constraint_d, diff_d in strategies:
            for _ in range(per_strategy):
                genome = BackdoorGenome()
                genome.randomize(constraint_d, diff_d)
                self.population.append(genome)

        # Fill remaining
        while len(self.population) < self.population_size:
            genome = BackdoorGenome()
            genome.randomize(random.uniform(0.03, 0.15), random.uniform(0.002, 0.006))
            self.population.append(genome)

        print(f"  Created {len(self.population)} genomes")

    def evaluate_genome(self, genome, num_batches=5):
        """Evaluate single genome using GPU batch testing"""
        matching, msg_A, msg_B, hash_A, hash_B = self.tester.test_condition(
            genome.msg_constraints,
            genome.diff_constraints,
            genome.correlations,
            num_batches=num_batches
        )

        genome.matching_bits = matching
        genome.msg_A = msg_A
        genome.msg_B = msg_B
        genome.hash_A = hash_A
        genome.hash_B = hash_B
        genome.is_collision = (matching == 256)

        constraint_count = np.sum(genome.msg_constraints != 0)
        genome.fitness = matching - 0.001 * constraint_count

        self.stats['evaluations'] += 1
        self.stats['hashes_computed'] += num_batches * self.batch_size * 2

        return genome

    def evaluate_population(self, num_batches=5):
        """Evaluate all genomes"""
        for genome in self.population:
            self.evaluate_genome(genome, num_batches)

    def select_parents(self, tournament_size=5):
        """Tournament selection"""
        parents = []
        for _ in range(self.population_size):
            tournament = random.sample(self.population, min(tournament_size, len(self.population)))
            winner = max(tournament, key=lambda g: g.fitness)
            parents.append(winner)
        return parents

    def create_next_generation(self):
        """Create next generation"""
        self.population.sort(key=lambda g: g.fitness, reverse=True)

        elite_count = max(2, self.population_size // 10)
        new_population = self.population[:elite_count]

        parents = self.select_parents()

        while len(new_population) < self.population_size:
            if random.random() < 0.7:
                p1, p2 = random.sample(parents, 2)
                child = p1.crossover(p2, self.generation + 1)
            else:
                parent = random.choice(parents)
                child = parent.mutate(0.1, self.generation + 1)

            if random.random() < 0.3:
                child = child.mutate(0.05, self.generation + 1)

            new_population.append(child)

        self.population = new_population[:self.population_size]
        self.generation += 1

    def inject_diversity(self):
        """Inject fresh genomes"""
        self.population.sort(key=lambda g: g.fitness, reverse=True)
        inject_count = self.population_size // 5

        for i in range(inject_count):
            genome = BackdoorGenome()
            genome.randomize(random.uniform(0.03, 0.12), random.uniform(0.002, 0.006))
            self.population[-(i+1)] = genome

    def run(self, max_generations=100, target_bits=256, batches_per_eval=8, verbose=True):
        """Run evolutionary search"""
        print("\n" + "="*70)
        print("  🚀 GPU-ACCELERATED EVOLUTIONARY DISCOVERY")
        print("="*70)
        print(f"  Population: {self.population_size}")
        print(f"  Batch size: {self.batch_size}")
        print(f"  Batches per eval: {batches_per_eval}")
        print(f"  Tests per generation: {self.population_size * batches_per_eval * self.batch_size:,}")

        self.initialize_population()

        start_time = time.time()
        no_improvement = 0
        last_best = 0

        for gen in range(max_generations):
            self.generation = gen
            gen_start = time.time()

            # Evaluate
            self.evaluate_population(num_batches=batches_per_eval)

            # Track best
            current_best = max(self.population, key=lambda g: g.matching_bits)

            if self.best_ever is None or current_best.matching_bits > self.best_ever.matching_bits:
                self.best_ever = BackdoorGenome()
                self.best_ever.msg_constraints = current_best.msg_constraints.copy()
                self.best_ever.diff_constraints = current_best.diff_constraints.copy()
                self.best_ever.matching_bits = current_best.matching_bits
                self.best_ever.fitness = current_best.fitness
                self.best_ever.msg_A = current_best.msg_A
                self.best_ever.msg_B = current_best.msg_B
                self.best_ever.hash_A = current_best.hash_A
                self.best_ever.hash_B = current_best.hash_B
                self.best_ever.is_collision = current_best.is_collision
                no_improvement = 0
            else:
                no_improvement += 1

            # Stats
            avg_matching = np.mean([g.matching_bits for g in self.population])
            max_matching = max(g.matching_bits for g in self.population)
            gen_time = time.time() - gen_start
            total_time = time.time() - start_time
            hashes_per_sec = self.stats['hashes_computed'] / total_time

            self.history.append({
                'generation': gen,
                'avg_matching': avg_matching,
                'max_matching': max_matching,
                'best_ever': self.best_ever.matching_bits,
                'time': gen_time
            })

            if verbose:
                summary = current_best.get_summary()
                print(f"\n  Gen {gen:3d} | Best: {max_matching:3d}/256 | "
                      f"Avg: {avg_matching:.1f} | Ever: {self.best_ever.matching_bits:3d} | "
                      f"{gen_time:.2f}s | {hashes_per_sec/1e6:.2f}M H/s")
                print(f"          | Constraints: {summary['msg_zeros']}z {summary['msg_ones']}o | "
                      f"Diffs: {summary['diff_bits']}")

            # Check collision
            if current_best.is_collision:
                print("\n" + "🚨"*30)
                print("  COLLISION FOUND! BACKDOOR CONFIRMED!")
                print("🚨"*30)
                self.stats['collisions_found'] += 1
                break

            if current_best.matching_bits >= target_bits:
                print(f"\n  Target {target_bits} bits reached!")
                break

            # Diversity injection
            if no_improvement >= 10:
                if verbose:
                    print(f"  [Injecting diversity - stuck for {no_improvement} gens]")
                self.inject_diversity()
                no_improvement = 0

            self.create_next_generation()

        self.stats['generations'] = self.generation
        self.stats['best_matching'] = self.best_ever.matching_bits if self.best_ever else 0

        # Final intensive search on best
        if self.best_ever and self.best_ever.matching_bits >= 200 and not self.best_ever.is_collision:
            print("\n  Running intensive search on best candidate...")
            self.intensive_search(self.best_ever, iterations=50)

        return self.best_ever

    def intensive_search(self, genome, iterations=50):
        """Intensive search on a promising genome"""
        best = genome

        for i in range(iterations):
            # Test with more batches
            matching, msg_A, msg_B, hash_A, hash_B = self.tester.test_condition(
                genome.msg_constraints,
                genome.diff_constraints,
                genome.correlations,
                num_batches=20
            )

            if matching > best.matching_bits:
                best.matching_bits = matching
                best.msg_A = msg_A
                best.msg_B = msg_B
                best.hash_A = hash_A
                best.hash_B = hash_B
                best.is_collision = (matching == 256)
                print(f"    Intensive search: {matching}/256")

                if matching == 256:
                    print("    COLLISION!")
                    break

            # Try small mutations
            mutant = genome.mutate(0.02)
            matching2, msg_A2, msg_B2, hash_A2, hash_B2 = self.tester.test_condition(
                mutant.msg_constraints,
                mutant.diff_constraints,
                mutant.correlations,
                num_batches=10
            )

            if matching2 > best.matching_bits:
                best = mutant
                best.matching_bits = matching2
                best.msg_A = msg_A2
                best.msg_B = msg_B2
                best.hash_A = hash_A2
                best.hash_B = hash_B2
                best.is_collision = (matching2 == 256)
                genome = mutant
                print(f"    Mutation improved: {matching2}/256")

        if best.matching_bits > self.best_ever.matching_bits:
            self.best_ever = best

    def analyze_best(self):
        """Analyze best found condition"""
        if not self.best_ever:
            print("No best genome found")
            return

        print("\n" + "="*70)
        print("  BEST DISCOVERED CONDITION")
        print("="*70)

        genome = self.best_ever

        print(f"\n  Matching bits: {genome.matching_bits}/256")
        print(f"  Is collision: {genome.is_collision}")

        # Constraint analysis
        msg_zeros = np.sum(genome.msg_constraints == 1)
        msg_ones = np.sum(genome.msg_constraints == 2)
        diff_bits = np.sum(genome.diff_constraints == 2)

        print(f"\n  CONSTRAINTS:")
        print(f"    Bits forced to 0: {msg_zeros}")
        print(f"    Bits forced to 1: {msg_ones}")
        print(f"    Differential bits: {diff_bits}")

        # Show pattern
        print(f"\n  MESSAGE CONSTRAINT PATTERN:")
        for word in range(16):
            constrained = []
            for bit in range(32):
                idx = word * 32 + bit
                if genome.msg_constraints[idx] == 1:
                    constrained.append(f"{bit}=0")
                elif genome.msg_constraints[idx] == 2:
                    constrained.append(f"{bit}=1")
            if constrained:
                print(f"    Word {word:2d}: {', '.join(constrained[:8])}{'...' if len(constrained) > 8 else ''}")

        print(f"\n  DIFFERENTIAL PATTERN:")
        for i in range(512):
            if genome.diff_constraints[i] == 2:
                word = i // 32
                bit = i % 32
                print(f"    Word {word}, Bit {bit}")

        print(f"\n  MESSAGES:")
        if genome.msg_A:
            print(f"    A: {genome.msg_A.hex()}")
        if genome.msg_B:
            print(f"    B: {genome.msg_B.hex()}")

        print(f"\n  HASHES:")
        if genome.hash_A:
            print(f"    H(A): {genome.hash_A}")
        if genome.hash_B:
            print(f"    H(B): {genome.hash_B}")


# ============================================================
# MULTI-RUN DISCOVERY
# ============================================================

class BackdoorHunterGPU:
    """Main interface for GPU-accelerated backdoor hunting"""

    def __init__(self):
        self.all_discoveries = []
        self.best_overall = None

    def run(self, generations=100, population=100, batch_size=2048,
            batches_per_eval=8, runs=3):
        """Run multiple discovery attempts"""

        print("\n" + "#"*70)
        print("#" + " "*10 + "🚀 GPU-ACCELERATED BACKDOOR DISCOVERY" + " "*13 + "#")
        print("#"*70)

        total_start = time.time()

        for run_idx in range(runs):
            print(f"\n\n{'='*70}")
            print(f"  DISCOVERY RUN {run_idx + 1}/{runs}")
            print(f"{'='*70}")

            # Clear GPU memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            engine = GPUBackdoorDiscovery(population_size=population, batch_size=batch_size)
            best = engine.run(
                max_generations=generations,
                batches_per_eval=batches_per_eval,
                verbose=True
            )

            if best:
                self.all_discoveries.append(best)

                if self.best_overall is None or best.matching_bits > self.best_overall.matching_bits:
                    self.best_overall = best

                if best.is_collision:
                    self._report_collision(best)
                    return best

        total_time = time.time() - total_start
        self._final_report(total_time)

        return self.best_overall

    def _report_collision(self, genome):
        """Report found collision"""
        print("\n\n" + "🚨"*35)
        print("  CRITICAL: BACKDOOR DISCOVERED!")
        print("🚨"*35)

        print(f"\n  COLLISION CONDITION:")
        print(f"    Bits forced to 0: {np.sum(genome.msg_constraints == 1)}")
        print(f"    Bits forced to 1: {np.sum(genome.msg_constraints == 2)}")
        print(f"    Differential bits: {np.sum(genome.diff_constraints == 2)}")

        print(f"\n  COLLIDING MESSAGES:")
        print(f"    A: {genome.msg_A.hex()}")
        print(f"    B: {genome.msg_B.hex()}")

        print(f"\n  SHARED HASH:")
        print(f"    {genome.hash_A}")

        # Verify with hashlib
        real_hash_A = hashlib.sha256(genome.msg_A).hexdigest()
        real_hash_B = hashlib.sha256(genome.msg_B).hexdigest()

        print(f"\n  VERIFICATION (hashlib):")
        print(f"    H(A): {real_hash_A}")
        print(f"    H(B): {real_hash_B}")
        print(f"    Match: {real_hash_A == real_hash_B}")

        with open("sha256_backdoor_found.json", "w") as f:
            json.dump(genome.to_dict(), f, indent=2)
        print(f"\n  Saved to: sha256_backdoor_found.json")

    def _final_report(self, total_time):
        """Final report"""
        print("\n\n" + "="*70)
        print("  DISCOVERY COMPLETE")
        print("="*70)

        print(f"\n  Total time: {total_time:.1f}s")
        print(f"  Runs completed: {len(self.all_discoveries)}")

        if self.best_overall:
            print(f"\n  BEST RESULT:")
            print(f"    Matching bits: {self.best_overall.matching_bits}/256")
            print(f"    Is collision: {self.best_overall.is_collision}")

            if self.best_overall.msg_A:
                print(f"\n  MESSAGES:")
                print(f"    A: {self.best_overall.msg_A.hex()}")
                print(f"    B: {self.best_overall.msg_B.hex()}")

            if self.best_overall.hash_A:
                print(f"\n  HASHES:")
                print(f"    H(A): {self.best_overall.hash_A}")
                print(f"    H(B): {self.best_overall.hash_B}")

            # Verify with hashlib
            if self.best_overall.msg_A and self.best_overall.msg_B:
                real_A = hashlib.sha256(self.best_overall.msg_A).hexdigest()
                real_B = hashlib.sha256(self.best_overall.msg_B).hexdigest()

                real_matching = sum(
                    8 - bin(a ^ b).count('1')
                    for a, b in zip(
                        bytes.fromhex(real_A),
                        bytes.fromhex(real_B)
                    )
                )

                print(f"\n  VERIFICATION (hashlib):")
                print(f"    Real matching bits: {real_matching}/256")

        # Verdict
        print("\n" + "-"*70)
        if self.best_overall and self.best_overall.is_collision:
            print("  🚨 VERDICT: BACKDOOR FOUND")
        elif self.best_overall and self.best_overall.matching_bits >= 200:
            print("  ⚠️  VERDICT: SUSPICIOUS - High correlation found")
        elif self.best_overall and self.best_overall.matching_bits >= 160:
            print("  ⚡ VERDICT: ANOMALY - Above random baseline")
        else:
            print("  ✓ VERDICT: No backdoor detected")

        # Save all
        if self.all_discoveries:
            data = {
                'best_matching': self.best_overall.matching_bits if self.best_overall else 0,
                'discoveries': [g.to_dict() for g in self.all_discoveries]
            }
            with open("sha256_gpu_discoveries.json", "w") as f:
                json.dump(data, f, indent=2)
            print(f"\n  Results saved to: sha256_gpu_discoveries.json")


# ============================================================
# MAIN
# ============================================================

def main():
    print("\n" + "="*70)
    print("  🚀 SHA-256 BACKDOOR DISCOVERY - GPU ACCELERATED")
    print("="*70)

    print(f"""
DEVICE: {device}
{'GPU: ' + torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU mode (slower)'}

SPEEDUP FACTORS:
• Batch SHA-256 on GPU: 100-1000x vs CPU sequential
• Parallel population evaluation
• Vectorized bit counting

CONFIGURATION OPTIONS:
• Population size: More genomes = better exploration
• Batch size: More messages tested per genome
• Generations: More evolution iterations

EXPECTED PERFORMANCE:
• GPU: ~1-10 million hashes/second
• CPU: ~10-100 thousand hashes/second
""")

    hunter = BackdoorHunterGPU()

    while True:
        print("\n" + "-"*50)
        print("OPTIONS:")
        print("  1. Quick search (50 gen, 50 pop, 2 runs)")
        print("  2. Standard search (100 gen, 100 pop, 3 runs)")
        print("  3. Deep search (200 gen, 150 pop, 5 runs)")
        print("  4. Intensive search (300 gen, 200 pop, 10 runs)")
        print("  5. Custom search")
        print("  q. Quit")

        choice = input("\nChoice: ").strip().lower()

        if choice == 'q':
            print("Goodbye!")
            break

        elif choice == '1':
            hunter.run(generations=50, population=50, batch_size=1024,
                      batches_per_eval=5, runs=2)

        elif choice == '2':
            hunter.run(generations=100, population=100, batch_size=2048,
                      batches_per_eval=8, runs=3)

        elif choice == '3':
            hunter.run(generations=200, population=150, batch_size=2048,
                      batches_per_eval=10, runs=5)

        elif choice == '4':
            hunter.run(generations=300, population=200, batch_size=4096,
                      batches_per_eval=12, runs=10)

        elif choice == '5':
            try:
                gens = int(input("Generations (default 100): ").strip() or "100")
                pop = int(input("Population (default 100): ").strip() or "100")
                batch = int(input("Batch size (default 2048): ").strip() or "2048")
                batches = int(input("Batches per eval (default 8): ").strip() or "8")
                runs = int(input("Runs (default 3): ").strip() or "3")

                hunter.run(generations=gens, population=pop, batch_size=batch,
                          batches_per_eval=batches, runs=runs)
            except ValueError:
                print("Invalid input")

        else:
            print("Invalid choice")


if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.6 MB/s eta 0:00:00
  🚀 SHA-256 BACKDOOR DISCOVERY - GPU ACCELERATED
  Device: cuda
  GPU: Tesla T4
  Memory: 15.8 GB

  🚀 SHA-256 BACKDOOR DISCOVERY - GPU ACCELERATED

DEVICE: cuda
GPU: Tesla T4

SPEEDUP FACTORS:
• Batch SHA-256 on GPU: 100-1000x vs CPU sequential
• Parallel population evaluation
• Vectorized bit counting

CONFIGURATION OPTIONS:
• Population size: More genomes = better exploration
• Batch size: More messages tested per genome
• Generations: More evolution iterations

EXPECTED PERFORMANCE:
• GPU: ~1-10 million hashes/second
• CPU: ~10-100 thousand hashes/second


--------------------------------------------------
OPTIONS:
  1. Quick search (50 gen, 50 pop, 2 runs)
  2. Standard search (100 gen, 100 pop, 3 runs)
  3. Deep search (200 gen, 150 pop, 5 runs)
  4. Intensive search (300 gen, 200 pop, 10 runs)
  5. Custom search
  q. Quit

Choice: 3

######################################################################